# Demo of EKF with Two-Stage Model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Two-stage model functions

This defines the bed of the Glacier given the paramter of slope, start point and reverse slope.

In [2]:
def bedtopo(L,*args):  #bed to topography L- length of glacier
    b0 = args[0]   #b0: glacier start point
    bx = args[1]   #bx: glacier slope
    sillmin = args[2]   #start of reverse slope
    sillmax = args[3]   #end of reverse slope
    sillslope = args[4]   #slope of reverse slope
    
    if L < sillmin: #if glacier is shorter than reverse slope
        b = b0 + bx*L
    elif L < sillmax: #if glacier terminates on reverse slope
        b = b0 + bx*sillmin + sillslope*(L-sillmin)
    else: #if glacier terminates after reverse slope
        b = b0 + bx*sillmin + sillslope*(sillmax-sillmin) + bx*(L-sillmax)
        
    return b

This function defines the model of the Glacier we are working with. The differential equations used are defined below in mathematical terms to help bring clarity to the code. 

In [3]:
def TwoStage_timedep(state,*args): #Robel et al. 2018 two-stage model
    
    Hnd, Lnd = state #Unpack the state vector, Hnd n-dim thickness
    #Lnd n-dim length
    f = np.zeros(2) #Derivatives
    
    smb0 = args[0] #P (precipitation) at time 0
    smb1 = args[1] #P (precipitation) at time 1
    smbf = args[2] #P (precipitation) at time f
    gamma = args[3] #Q coefficient
    omega = args[4] #Q_g coefficient
    time = args[5]
    t1 = args[6]
    tfinal = args[7]
#    b0 = args[3]
#    bx = args[4]
#    sillmin = args[5]
#    sillmax = args[6]
#    sillslope = args[7]

    b0 = 0   #glacier start point
    bx = args[11]   #glacier slope
    sillmin = args[8]*1e3   #reverse slope start point
    sillmax = args[9]*1e3   #reverse slope ends point
    sillslope = args[10]    #reverse slope slope
    
    rhow = 1028   #density of water
    rhoi = 917   #denisty of ice
    n = 3   
    beta = 4.75 
    Lscale = 100e3   #scale for length, it is on the order of 10E4 or 1E5
    Hscale = 1000   #scale for thickness, it is on the order of 10E2 or 1E3
    
    H = Hnd*Hscale #icethickness
    L = Lnd*Lscale #length of glacier
    
    if time < t1:
        smb = smb0 + (smb1-smb0)*time/t1
    else:
        smb = smb1 + (smbf-smb1)*time/(tfinal-t1) 
    #see below for write out of the functions in mathematical notation
    hg = -(rhow/rhoi)*bedtopo(L,b0,bx,sillmin,sillmax,sillslope)   # #1
    Q = gamma * (H**(2*n + 1))/(L**n)   # #2
    Qg = omega * (hg**beta)   # #3
    
    f[0] = (smb - (Qg/L) - (H/(hg*L))*(Q-Qg))/Hscale   # #4
    f[1] = (Q-Qg)/hg/Lscale   # #5
    return f 

$\#1$ $h_g = \dfrac{\rho_w}{\rho_i}b(L)$

$\#2$ $Q=\gamma \dfrac{H^{2n+1}}{L^n}$

$\#3$ $Q_g= \Omega h_g^\beta$

$\#4$ $\dfrac{dh}{dt}=P-\dfrac{Q_g}{L}-\dfrac{H}{h_gL}(Q-Q_g)$

$\#5$ $\dfrac{dL}{dt}=\dfrac{1}{h_g}(Q-Q_g) $ Note: You can factor out scalars of a derivative, that is why it is able to be divide by the scale of L.

### Time Integrator function

This function defines and runs a 4th Order Runge Kutta Method.

In [4]:
def RK4(rhs,state,dt,*args): #ask for explanation
    
    k1 = rhs(state,*args)
    k2 = rhs(state+k1*dt/2,*args)
    k3 = rhs(state+k2*dt/2,*args)
    k4 = rhs(state+k3*dt,*args)

    new_state = state + (dt/6)*(k1+2*k2+2*k3+k4)
    return new_state

### EKF functions

This function runs the Ensemble Kalman Filter.

In [5]:
def EnKF(ubi,w,ObsOp,JObsOp,R,B):
    
    # The analysis step for the (stochastic) ensemble Kalman filter 
    # with virtual observations

    n,N = ubi.shape # n is the state dimension and N is the size of ensemble
    m = w.shape[0] # m is the size of measurement vector

    # compute the mean of forecast ensemble
    ub = np.mean(ubi,1)
    # compute Jacobian of observation operator at ub
    Dh = JObsOp(w)
    # compute Kalman gain
    D = Dh@B@Dh.T + R #@ is matrix multiplication
    K = B @ Dh.T @ np.linalg.inv(D)
        
    
    wi = np.zeros([m,N])
    uai = np.zeros([n,N])
    for i in range(N):
        # create virtual observations
        wi[:,i] = w + np.random.multivariate_normal(np.zeros(m), R)
        # compute analysis ensemble
        uai[:,i] = ubi[:,i] + K @ (wi[:,i]-ObsOp(ubi[:,i]))
        
    # compute the mean of analysis ensemble
    ua = np.mean(uai,1)    
    # compute analysis error covariance matrix
    P = (1/(N-1)) * (uai - ua.reshape(-1,1)) @ (uai - ua.reshape(-1,1)).T
    return uai, P

Defines the observation operators we will be using.

In [6]:
# Observation operators
def h(u):   #identity oberservation operator?
    w = u
    return w

def Dh(u):   #identity operator for matrix of size Dh
    n = len(u)
    D = np.eye(n)
    return D

## Set parameters

Sets model intialization parameters except for length of glacier, glacier bed parameters and reverse slope parameters.

In [7]:
#set possible parameter values
smb_t1 = [1950]
smb_tf = [2300]
smb_0d = [0.3]
smb_1d = [0.15]
smb_fd = [0.0]
sillmin_d = [415]
sillmax_d = [425]
sillslope_d = [0.001]
hnd_d = [2.3]
lnd_d = [4.6]
bx_d = [-0.001]
#create array of zeros to store possible values 
arr=np.zeros((len(smb_t1),len(smb_tf),len(smb_0d),len(smb_1d),\
              len(smb_fd), len(sillmin_d),len(sillmax_d),len(sillslope_d),\
            len(hnd_d),len(lnd_d),len(bx_d)),dtype=list)
#series of nested for loops to build lists of parameters into the array
for a2 in range(len(smb_t1)):
    for a3 in range(len(smb_tf)):
        for a4 in range(len(smb_0d)):
            for a5 in range(len(smb_1d)):
                for a6 in range(len(smb_fd)):
                    for a7 in range(len(sillmin_d)):
                        for a8 in range(len(sillmax_d)):
                            for a9 in range(len(sillslope_d)):
                                for a10 in range(len(hnd_d)):
                                    for a11 in range(len(lnd_d)):
                                        for a12 in range(len(bx_d)):
                                            #note that the number of brackets denotes corresponds to
                                            #the number of dimensions
                                            arr[a2][a3][a4][a5][a6][a7][a8][a9][a10][a11][a12]=\
                                            [smb_t1[a2],smb_tf[a3],smb_0d[a4],smb_1d[a5],smb_fd[a6],\
                                            sillmin_d[a7],sillmax_d[a8],sillslope_d[a9],hnd_d[a10],lnd_d[a11],bx_d[a12]]

In [8]:
#runs the truth simulation generation for all the cases
for a2 in range(len(smb_t1)):
    for a3 in range(len(smb_tf)):
        for a4 in range(len(smb_0d)):
            for a5 in range(len(smb_1d)):
                for a6 in range(len(smb_fd)):
                    for a7 in range(len(sillmin_d)):
                        for a8 in range(len(sillmax_d)):
                            for a9 in range(len(sillslope_d)):
                                for a10 in range(len(hnd_d)):
                                    for a11 in range(len(lnd_d)):
                                        for a12 in range(len(bx_d)):
                                            var = arr[a2][a3][a4][a5][a6][a7][a8][a9][a10][a11][a12]
                                            nd = 2 #degrees of freedom (number of prognositic equations in model)
                                            # parameters
                                            smb0 = var[2]
                                            smb1 = var[3]
                                            smbf = var[4]
                                            t1 = var[0]
                                            gamma = 0.05
                                            omega = 8e-8
                                            #b0 = 0
                                            bx = var[10]
                                            #sillmin = 430e3
                                            #sillmax = 440e3
                                            #sillslope = 0.01
                                            sillmin=var[5]
                                            sillmax=var[6]
                                            sillslope=var[7]

                                            dt = 0.25
                                            tm = var[1]
                                            nt = int(tm/dt)
                                            t = np.linspace(0,tm,nt+1)
                                            u0True = np.array([var[8],var[9]]) # True initial conditions
                                            # first is H, second is L
                                            #np.random.seed(seed=1)
                                            sig_m_acc= 0.1  # standard deviation for accurate measurement noise
                                            sig_m_inacc= 0.07  # standard deviation for inaccurate measurement noise

                                            ind_m_inacc = np.linspace(200,1800,9).astype(int) #np.array([1000,1900])
                                            ind_m_acc = np.linspace(2001,2020,20).astype(int)
                                            ind_m = np.concatenate((ind_m_inacc,ind_m_acc))
                                            t_m = t[ind_m]
                                            nt_m = np.size(ind_m)
                                            sig_m =  np.zeros(nt+1)
                                            sig_m[ind_m_inacc] = sig_m_inacc
                                            sig_m[ind_m_acc] = sig_m_acc

                                            #time integration
                                            uTrue = np.zeros([nd,nt+1])
                                            uTrue[:,0] = u0True
                                            km = 0
                                            w = np.zeros([nd,nt_m])
                                            for k in range(nt):
                                                time = k*dt
                                                uTrue[:,k+1] = RK4(TwoStage_timedep,uTrue[:,k],dt,smb0,smb1,smbf,gamma,omega,time,t1,tm,sillmin,sillmax,sillslope,bx)
                                                if (km<nt_m) and (k+1==ind_m[km]):
                                                    w[:,km] = h(uTrue[:,k+1]) + np.random.normal(0,sig_m[k+1],[nd,])
                                                    km = km+1
                                            # saves into a csv
                                            dft=pd.DataFrame(t.T, columns=['t'])
                                            df=pd.DataFrame(uTrue.T, columns=['H','L'])
                                            data1=df.join(dft)
                                            data=data1.set_index('t')
                                            data.to_csv(f"data_single_cases/truth_simulation/smbt1{str(var[0])}smbtf{var[1]}smb0{str(var[2])}smb1{str(var[3])}smbf{str(var[4])}sillmin{str(var[5])}sillmax{str(var[6])}sillslope{str(var[7])}hnd{str(var[8])}lnd{str(var[9])}bx{str(var[10])}.csv")